In [1]:
'''
文本相似度分析的步骤
1、读取文档
2、对要计算的多篇文档进行分词
3、对文档进行整理成指定格式，方便后续进行计算
4、计算出词语的词频
5、【可选】对词频低的词语进行过滤
6、建立语料库词典
7、加载要对比的文档
8、将要对比的文档通过doc2bow转化为词袋模型
9、对词袋模型进行进一步处理，得到新语料库
10、将新语料库通过tfidfmodel进行处理，得到tfidf
11、通过token2id得到特征数
12、稀疏矩阵相似度，从而建立索引
13、得到最终相似度结果
'''

'\n文本相似度分析的步骤\n1、读取文档\n2、对要计算的多篇文档进行分词\n3、对文档进行整理成指定格式，方便后续进行计算\n4、计算出词语的词频\n5、【可选】对词频低的词语进行过滤\n6、建立语料库词典\n7、加载要对比的文档\n8、将要对比的文档通过doc2bow转化为词袋模型\n9、对词袋模型进行进一步处理，得到新语料库\n10、将新语料库通过tfidfmodel进行处理，得到tfidf\n11、通过token2id得到特征数\n12、稀疏矩阵相似度，从而建立索引\n13、得到最终相似度结果\n'

In [1]:
import jieba
from gensim import corpora,models,similarities

In [2]:
doc0 = "我不喜欢上海"
doc1 = "上海是一个好地方"
doc2 = "北京是一个好地方"
doc3 = "上海好吃的在哪里"
doc4 = "上海好玩的在哪里"
doc5 = "上海是好地方"
doc6 = "上海路和上海人"
doc7 = "喜欢小吃"
doc_test="我喜欢上海的小吃"

In [3]:
all_doc = []
all_doc.append(doc0)
all_doc.append(doc1)
all_doc.append(doc2)
all_doc.append(doc3)
all_doc.append(doc4)
all_doc.append(doc5)
all_doc.append(doc6)
all_doc.append(doc7)

In [4]:
all_doc_list = []
for doc in all_doc:  
    doc_list = [word for word in jieba.cut(doc)]  
    all_doc_list.append(doc_list)
all_doc_list

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/dm/n5w6s3vs0yv6nm2h3smdvs380000gn/T/jieba.cache
Loading model cost 0.792 seconds.
Prefix dict has been built successfully.


[['我', '不', '喜欢', '上海'],
 ['上海', '是', '一个', '好', '地方'],
 ['北京', '是', '一个', '好', '地方'],
 ['上海', '好吃', '的', '在', '哪里'],
 ['上海', '好玩', '的', '在', '哪里'],
 ['上海', '是', '好', '地方'],
 ['上海', '路', '和', '上海', '人'],
 ['喜欢', '小吃']]

In [5]:
doc_test_list = [word for word in jieba.cut(doc_test)]
doc_test_list

['我', '喜欢', '上海', '的', '小吃']

In [6]:
# 用dictionary方法获取词袋
dictionary = corpora.Dictionary(all_doc_list)
dictionary.keys(), dictionary.token2id

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
 {'上海': 0,
  '不': 1,
  '喜欢': 2,
  '我': 3,
  '一个': 4,
  '地方': 5,
  '好': 6,
  '是': 7,
  '北京': 8,
  '哪里': 9,
  '在': 10,
  '好吃': 11,
  '的': 12,
  '好玩': 13,
  '人': 14,
  '和': 15,
  '路': 16,
  '小吃': 17})

In [7]:
# 使用doc2bow制作语料库
corpus = [dictionary.doc2bow(doc) for doc in all_doc_list]

In [8]:
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(0, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1)],
 [(0, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(0, 1), (9, 1), (10, 1), (12, 1), (13, 1)],
 [(0, 1), (5, 1), (6, 1), (7, 1)],
 [(0, 2), (14, 1), (15, 1), (16, 1)],
 [(2, 1), (17, 1)]]

In [9]:
doc_test_vec = dictionary.doc2bow(doc_test_list)
doc_test_vec

[(0, 1), (2, 1), (3, 1), (12, 1), (17, 1)]

In [10]:
# 相似度分析：使用TF-IDF模型对语料库建模
tfidf = models.TfidfModel(corpus)

In [11]:
# 获取测试文档中，每个词的TF-IDF值
tfidf[doc_test_vec]

[(0, 0.08112725037593049),
 (2, 0.3909393754390612),
 (3, 0.5864090631585919),
 (12, 0.3909393754390612),
 (17, 0.5864090631585919)]

In [12]:
# 对每个目标文档，分析测试文档的相似度
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=len(dictionary.keys()))
sim = index[tfidf[doc_test_vec]]
sim

array([0.54680777, 0.01055349, 0.        , 0.17724207, 0.17724207,
       0.01354522, 0.01279765, 0.70477605], dtype=float32)

In [13]:
# 根据相似度排序：
# 从分析结果来看，测试文档与doc7相似度最高，其次是doc0，与doc2的相似度为零。大家可以根据TF-IDF的原理，看看是否符合预期。
sorted(enumerate(sim), key=lambda item: -item[1])

[(7, 0.70477605),
 (0, 0.54680777),
 (3, 0.17724207),
 (4, 0.17724207),
 (5, 0.013545224),
 (6, 0.01279765),
 (1, 0.010553493),
 (2, 0.0)]

In [14]:
sorted(enumerate(sim), key=lambda item: item[1])

[(2, 0.0),
 (1, 0.010553493),
 (6, 0.01279765),
 (5, 0.013545224),
 (3, 0.17724207),
 (4, 0.17724207),
 (0, 0.54680777),
 (7, 0.70477605)]